# Introduction to Crunchers

In this notebook we demonstrate crunchers, silicone's most fundamental infilling methods. You will need to install silicone from as described in the readme in order to run this code. It introduces one specific cruncher, the 'closest RMS' cruncher, and the basic tools to manipulate it. Other crunchers will be documented in other notebooks. 

## Imports

In [1]:
import os.path
import traceback

import pandas as pd
import pyam
import matplotlib.pyplot as plt
import numpy as np

import silicone.database_crunchers
from silicone.utils import (
    _get_unit_of_variable,
    find_matching_scenarios,
    _make_interpolator,
    _make_wide_db, 
    get_sr15_scenarios
)
import silicone.scripts

<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler


ModuleNotFoundError: No module named 'silicone.scripts'

## Assembling example data

Here we pull some example data by downloading it from the IAMC 1.5°C Scenario Explorer and Data hosted by IIASA (for details, see Huppman et al 2019, Integrated Assessment Modeling Consortium & International Institute for Applied Systems Analysis, 2018). If the data has already been downloaded before, we will use that instead for brevity, handled by a convenient wrapper function. We select only a few cases. 

In [ ]:
valid_model_ids = [
        "MESSAGE*",
        "AIM*",
        "C-ROADS*",
        "GCAM*",
        #"IEA*",
        #"IMAGE*",
        #"MERGE*",
        #"POLES*",
        #"REMIND*",
        "WITCH*"
    ]
SR15_SCENARIOS = "./sr15_scenarios.csv"
sr15_data = download_or_load_sr15(SR15_SCENARIOS, valid_model_ids)

### Starting point

Our starting point is the test data, loaded as a `pyam.IamDataFrame`. It may be helpful for understanding some of this tutorial for you to know more about IamDataFrames, documented at https://pyam-iamc.readthedocs.io/en/latest/. The key functions are `.filter()`, which selects a subset of data, and `.timeseries()`, which restructures the data into columns by date, with long indexes containing information other than the value. 

In [ ]:
sr15_data = pyam.IamDataFrame(SR15_SCENARIOS)
sr15_data.timeseries().head()

## Crunchers

Silicone's 'crunchers' are used to determine the relationship between a 'follower variable' and 'lead variable(s)' from a given database. The 'follower variable' is the variable for which we want to generate data e.g. `Emissions|C3F8` while the 'lead variable(s)' is the variable we want to use in order to infer a timeseries of the 'follower variable'. The lead variable is currently always a list with only a single item, e.g. `[Emissions|HFC]` but in future may be able to deal with several elements. The follower is always a single string - to infill multiple values in a collective way (e.g. splitting HFC into its various components) see Multiple Infillers. 

Each cruncher needs to be initialised with a database and has a `derive_relationship` method, which returns the infilling function. Its docstring describes what it does.

In [ ]:
print(silicone.database_crunchers.base._DatabaseCruncher.derive_relationship.__doc__)
# # an alternative, pop-up interface is shown if you uncomment the line below
#silicone.database_crunchers.base._DatabaseCruncher.derive_relationship?

These crunchers are best explored by looking at the examples below.

### Closest Root Mean Square cruncher

This cruncher finds the scenario and model in the infiller database that minimises the RMS distance between the the lead emissions in the infiller and infillee data. The follow data is then directly taken from that scenario. 
The full documentation is as follows:

In [ ]:
print(silicone.database_crunchers.DatabaseCruncherRMSClosest.__doc__)

#### Infilling

Firstly, let's cut the database down to a size that is comprehensible.

In [ ]:
sr15_data_closest_rms = sr15_data.filter(model=["WITCH-GLOBIOM 4.2"])
sr15_data_closest_rms.head()

Next, we initialise the type of cruncher we want with the infiller database. 

In [ ]:
cruncher = silicone.database_crunchers.DatabaseCruncherRMSClosest(sr15_data_closest_rms)

Now we can derive the relationship between the follower and leader, here `Emissions|CO2` and `Emissions|VOC` respectively. This returns a infiller function, which we will use later. Firstly we will check out the docstring. 

In [ ]:
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"])
print(filler.__doc__)
# filler info:

Now we can use this relationship to do some infilling. As a sanity check, we firstly make sure that if we pass in a CO$_2$ timeseries which is already in the database, we get back its VOC emissions timeseries. 

In [ ]:
example_model_scen = {
    "model": "WITCH-GLOBIOM 4.2",
    "scenario": "ADVANCE_INDC",
}
example_input = sr15_data_closest_rms.filter(**example_model_scen).data
example_input["model"] = "example"
example_input["scenario"] = "example"
example_input = pyam.IamDataFrame(example_input)

We can now use the function we returned above to infill the example data, and see that we recover the data we put in. 

In [ ]:
example_input_filled = filler(example_input)
example_input_filled.timeseries()

In [ ]:
sr15_data_closest_rms.filter(variable="Emissions|VOC", **example_model_scen).timeseries()

As expected, the cruncher has picked out the scenario which matches and returned its VOC timeseries. This can also be seen in a plot below, where the dashed (infilled) line perfectly matches the pink line. 

In [ ]:
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
sr15_data_closest_rms.filter(variable="*CO2").line_plot(ax=ax, **pkwargs)
example_input.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15), label="To infill")
ax.set_title("Database CO$_2$ and lead emissions timeseries")

ax = fig.add_subplot(122)
sr15_data_closest_rms.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
example_input_filled.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15), label="Infilled")
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

Now we can use our filler to infill other timeseries not present in our infiller database.

In [ ]:
filler_input = sr15_data.filter(model="MESSAGEix-GLOBIOM 1.0", scenario="CD-LINKS_NPi2020_1600")
# This scenario is missing `Emissions|VOC`. 
filler_input.filter(variable="Emissions|VOC").data

In [ ]:
# It can go in the same infiller we defined previously.
filler_input_filled = filler(filler_input)

In [ ]:
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
sr15_data_closest_rms.filter(variable="*CO2").line_plot(ax=ax, **pkwargs)
filler_input.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15), label="To infill")
ax.set_title("Database CO$_2$ and lead emissions timeseries")

ax = fig.add_subplot(122)
sr15_data_closest_rms.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
filler_input_filled.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15), label="Infilled")
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

As we can see, on an RMS basis our input timeseries is closest to the 'ADVANCE_2030_Med2C' scenario and hence its `Emissions|VOC` pathway is returned.

In [ ]:
filler_input_filled.timeseries()

In [ ]:
sr15_data_closest_rms.filter(scenario="ADVANCE_2030_Med2C", variable="Emissions|VOC").timeseries()

This may be regarded as conceptually the simplest of the data-based crunchers. The other crunchers are introduced in other notebooks. 